In [ ]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [ ]:
filepath = Path("../sample_files/lvhexmesh_typeA.vtk")
lv = LV.from_file(filepath)
lv.plot(show_edges=True)

In [ ]:
from project_heart.utils.spatial_utils import *
from project_heart.utils.vector_utils import *

In [ ]:
lv.identify_base_and_apex_regions()
lv.identify_epi_endo_regions(85)
lv.plot(scalars=LV_MESH_DATA.EPI_ENDO_GUESS.value, categorical=True)

In [ ]:
e1=15
b1 = 1.25
b2 = 100

endo_epi = np.copy(lv.get(GEO_DATA.SURF_POINT_DATA, LV_MESH_DATA.EPI_ENDO_GUESS))

lvsurf = lv.get_surface_mesh()
# Get edges
edges = lv.mesh.extract_feature_edges(e1)
edges = edges.extract_largest()
edges = edges.extract_largest()
edge_pts = edges.points
est_base = centroid(edge_pts)
est_radius = radius(edge_pts)
# select pts close to est_base based on % of est_radius
pts = lvsurf.points
d_base = np.linalg.norm(pts - est_base, axis=1)
ioi = np.where(d_base <= est_radius*b1)[0]
# re-estimate base centroid and radius
poi = pts[ioi]
# filter selected pts based on surface angle
surfnormals = lvsurf.compute_normals(inplace=True)
surf_normals = lvsurf.get_array("Normals", "points")
base_vecs = est_base - poi
base_angles = angle_between(surf_normals[ioi], base_vecs, check_orientation=False)
ioi = ioi[np.where(base_angles <= np.radians(b2) )[0]]
# filter by endo
ioi = ioi[np.where(endo_epi[ioi] != LV_SURFS.ENDO)]

# center_pts = pts[ioi]
# est_base = centroid(center_pts)
# est_radius = radius(center_pts)
# lv.add_virtual_node(LV_VIRTUAL_NODES.MITRAL, est_base)

# identify final surfaces
endo_epi_base = np.copy(endo_epi)
endo_epi_base[ioi] = LV_SURFS.BASE
# map to 'global' mesh ids
id_map = lv.get_surface_id_map_from_mesh()
endo_epi_base_mesh = np.zeros(lv.mesh.n_points)
endo_epi_base_mesh[id_map] = endo_epi_base
# add data to mesh
lv.surface_mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base
lv.mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base_mesh

In [ ]:
#plot
lv.plot("mesh", 
        LV_MESH_DATA.SURFS.value,
        vnodes=[
          (LV_VIRTUAL_NODES.APEX, "green"),
          (LV_VIRTUAL_NODES.BASE, "red"),
          (LV_VIRTUAL_NODES.MITRAL, "purple"),
          
          ],
        categorical=True)